In [1]:
# by Joh Schoeneberg, 10/2019


In [1]:
%matplotlib inline
import numpy as np
import skimage.external.tifffile
import os
import matplotlib.pyplot as plt
from PIL import Image

import multiprocessing  
from multiprocessing import Pool


#import sys
#    sys.path.append('/Users/johannesschoeneberg/git/JohSchoeneberg/pyLattice/src/python/pylattice/functions')
#import ParallelFunctions

In [2]:


inputFolder = '/Users/johannesschoeneberg/Desktop/PostDoc/Drubin_Betzig_Project/Janelia/Janelia_visit_2019-08-05_to_16/data/correct_deconvolution_artifacts/20190812_Imaging__Region_6/'
outputFolder = '/Users/johannesschoeneberg/Desktop/PostDoc/Drubin_Betzig_Project/Janelia/Janelia_visit_2019-08-05_to_16/data/correct_deconvolution_artifacts/20190812_Imaging__Region_6/output/'


In [3]:
# prepare the input data for the worker pool

parameters_for_workers = []
for file in os.listdir(inputFolder):
    if file.endswith(".tif"):
        parameters_for_workers.append((inputFolder,outputFolder,file))
print(parameters_for_workers)

[('/Users/johannesschoeneberg/Desktop/PostDoc/Drubin_Betzig_Project/Janelia/Janelia_visit_2019-08-05_to_16/data/correct_deconvolution_artifacts/20190812_Imaging__Region_6/', '/Users/johannesschoeneberg/Desktop/PostDoc/Drubin_Betzig_Project/Janelia/Janelia_visit_2019-08-05_to_16/data/correct_deconvolution_artifacts/20190812_Imaging__Region_6/output/', 'Region_6_Iter_0000_CamA_ch0_CAM1_stack0000_488nm_0000000msec_0000163955msecAbs_000x_000y_000z_0000t_decon.tif'), ('/Users/johannesschoeneberg/Desktop/PostDoc/Drubin_Betzig_Project/Janelia/Janelia_visit_2019-08-05_to_16/data/correct_deconvolution_artifacts/20190812_Imaging__Region_6/', '/Users/johannesschoeneberg/Desktop/PostDoc/Drubin_Betzig_Project/Janelia/Janelia_visit_2019-08-05_to_16/data/correct_deconvolution_artifacts/20190812_Imaging__Region_6/output/', 'Region_6_Iter_0001_CamA_ch0_CAM1_stack0000_488nm_0000000msec_0000296742msecAbs_000x_000y_000z_0001t_decon.tif'), ('/Users/johannesschoeneberg/Desktop/PostDoc/Drubin_Betzig_Project/

In [4]:
def open3dTiff(path):

    img = Image.open(path)

    numFrames = img.n_frames
    imgArray = np.zeros( ( numFrames,img.size[1], img.size[0] ),np.uint16 )

    for i in range(0,numFrames):
        img.seek(i)
        imgArray[i,:,:] = np.array(img)

    return imgArray

In [8]:
def openFile(inputFolder,outputFolder,file):
    inputPath = os.path.join(inputFolder, file)

    #latticeMovieStack = ParallelFunctions.open3dTiff(inputPath)
    latticeMovieStack = open3dTiff(inputPath)
    #print(latticeMovieStack.shape)

    #latticeMovieStack_processed = cropLatticeFrame_core(latticeMovieStack,center, margin)
#
    ##output_folder = Path(outputFolder)
    #outputPath = os.path.join(outputFolder, file)
    ##print(outputPath)
    #skimage.external.tifffile.imsave(outputPath,latticeMovieStack_processed.astype('uint16'))

    return latticeMovieStack

In [9]:
def correct_decon_boundary_artifact_for_single_slice(oneSlice,margin=20,verbose=False):

    summed = np.sum(oneSlice,axis=0)
    all_zero = np.where(summed==0)[0]
    all_nonZero = np.where(summed!=0)[0]
    
    middle = np.average(np.where(summed!=0))
    leftBoundary = np.max(all_zero[all_zero<middle])
    rightBoundary = np.min(all_zero[all_zero>middle])

    
    #correctedSlice = np.copy(stack[sliceIndex])
    correctedSlice = oneSlice
    correctedSlice[:,0:leftBoundary+margin]=0
    correctedSlice[:,rightBoundary-margin:correctedSlice.shape[1]]=0
    
    
    if(verbose):
        print("0:"+str(leftBoundary+margin))
        print(str(rightBoundary-margin)+":"+str(correctedSlice.shape[1]))
        
        plt.figure(dpi=150)
        plt.plot(summed,color='grey')
        #plt.axvline(middle,color='red')
        plt.axvline(leftBoundary, color='black')
        plt.axvline(rightBoundary, color='black')
        
        plt.axvline(leftBoundary+margin, color='blue')
        plt.axvline(rightBoundary-margin, color='blue')
        plt.show()
        
        plt.figure(dpi=150)
        plt.imshow(correctedSlice)
    
    return(correctedSlice)

In [10]:
verbose = True

for parameterSet in parameters_for_workers:
    stack = openFile(parameterSet[0],parameterSet[1],parameterSet[2])
    
    if(verbose):
        inputPath = parameterSet[0]
        print(inputPath)
        outputPath =parameterSet[1]
        print(outputPath)
        file = parameterSet[2]
        print(file)
    
    for i in range(0,stack.shape[0]):
        #remove the first and last slices from the new stack
        oneSlice = stack[i]
        if (i == 0 or i == stack.shape[0]-1):
            stack[i] = oneSlice[:,:]=0
        else:
            stack[i] = correct_decon_boundary_artifact_for_single_slice(oneSlice,verbose=False)
    skimage.external.tifffile.imsave(outputPath+file+'_deconArtifactCorrected.tif',stack.astype('uint16'))


/Users/johannesschoeneberg/Desktop/PostDoc/Drubin_Betzig_Project/Janelia/Janelia_visit_2019-08-05_to_16/data/correct_deconvolution_artifacts/20190812_Imaging__Region_6/
/Users/johannesschoeneberg/Desktop/PostDoc/Drubin_Betzig_Project/Janelia/Janelia_visit_2019-08-05_to_16/data/correct_deconvolution_artifacts/20190812_Imaging__Region_6/output/
Region_6_Iter_0000_CamA_ch0_CAM1_stack0000_488nm_0000000msec_0000163955msecAbs_000x_000y_000z_0000t_decon.tif
/Users/johannesschoeneberg/Desktop/PostDoc/Drubin_Betzig_Project/Janelia/Janelia_visit_2019-08-05_to_16/data/correct_deconvolution_artifacts/20190812_Imaging__Region_6/
/Users/johannesschoeneberg/Desktop/PostDoc/Drubin_Betzig_Project/Janelia/Janelia_visit_2019-08-05_to_16/data/correct_deconvolution_artifacts/20190812_Imaging__Region_6/output/
Region_6_Iter_0001_CamA_ch0_CAM1_stack0000_488nm_0000000msec_0000296742msecAbs_000x_000y_000z_0001t_decon.tif
/Users/johannesschoeneberg/Desktop/PostDoc/Drubin_Betzig_Project/Janelia/Janelia_visit_201